In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [3]:
games= pd.read_csv("Data//game.csv")
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21587 entries, 0 to 21586
Data columns (total 12 columns):
player          21587 non-null object
color           21587 non-null object
opponent        21587 non-null object
player_Elo      20555 non-null float64
opponent_Elo    21266 non-null float64
result          21587 non-null object
event           20811 non-null object
site            21556 non-null object
date            21587 non-null object
lines           21392 non-null object
moves           21392 non-null float64
file_name       21587 non-null object
dtypes: float64(3), object(9)
memory usage: 2.0+ MB


In [4]:
games.head()


,player,color,opponent,player_Elo,opponent_Elo,result,event,site,date,lines,moves,file_name
0,Tal,Black,Feibert Fred (GER),NaN,2285.0,Win,It (open),Leningrad (Russia),1991.??.??,1. d4 Nf6 2. Nf3 c5 3. c3 e6 4. Bf4 b6 5. e3 B...,63.0,Feibert-Fred_vs_Tal-Mikhail_1991.__.__.pgn
1,Tal,Black,Westerinen Heikki M J (FIN),NaN,2410.0,Win,It,Moscow (Russia),1982.??.??,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...,74.0,Westerinen-Heikki-M-J_vs_Tal-Mikhail_1982.__._...
2,Tal,Black,Larsen Bent (DEN),NaN,2565.0,Win,It (cat.14),Bugojno (Bosnia & Herzegovina),1984.??.??,1. c4 c5 2. g3 g6 3. Bg2 Bg7 4. e3 Nf6 5. Nc3 ...,56.0,Larsen-Bent_vs_Tal-Mikhail_1984.__.__.pgn
3,Tal,White,Azmaiparashvili Zurab (GEO),NaN,2470.0,Draw,Memorial V.Goglidze,Tbilisi (Georgia),1986.??.??,1. d4 d6 2. Nf3 g6 3. g3 Bg7 4. Bg2 Nd7 5. O-O...,31.0,Tal-Mikhail_vs_Azmaiparashvili-Zurab_1986.__._...
4,Tal,White,Wedberg Tom (SWE),NaN,2480.0,Win,It (open),New York (USA),1990.??.??,1. Nf3 Nf6 2. c4 e6 3. g3 d5 4. Bg2 Be7 5. O-O...,69.0,Tal-Mikhail_vs_Wedberg-Tom_1990.__.__.pgn


In [5]:
games=games.drop(columns=['file_name','lines'])

In [6]:
games=games[games.player != 'Alekhine']
games=games[games.player != 'Capablanca']
games=games[games.player != 'Tal']
games=games[games.player != 'Morphy']
games=games[games.player != 'Polgar']
games

,player,color,opponent,player_Elo,opponent_Elo,result,event,site,date,moves
2672,Polgar,White,"Pashikian, A.",2693.0,2612.0,Draw,15th ch-EUR Indiv 2014,Yerevan ARM,2014.03.07,95.0
2673,Polgar,White,Ftacnik Lubomir (SVK),2630.0,2535.0,Win,Zt,Budapest (Hungary),1993.??.??,77.0
2674,Polgar,Black,Leko Peter (HUN),2665.0,2630.0,Draw,It (cat.17),Leon (Spain),1996.??.??,71.0
2675,Polgar,White,Leko Peter (HUN),2665.0,2630.0,Lose,It (cat.16),Tilburg (Netherlands),1996.??.??,122.0
2676,Polgar,Black,"Karpov, Ana",2708.0,2670.0,Win,Chess Champions League,Vitoria Gasteiz ESP,2007.11.10,86.0
...,...,...,...,...,...,...,...,...,...,...
20308,Anand,White,"Aronian, L.",2799.0,2739.0,Draw,Amber Rapid,Nice FRA,2008.03.23,57.0
20309,Anand,White,Adams Michael (ENG),2725.0,2630.0,Draw,Ch World (qual) (PCA),Groningen (Netherlands),1993.??.??,108.0
20310,Anand,Black,"Korobov, A.",2779.0,2692.0,Draw,World Blitz 2016,Doha QAT,2016.12.29,66.0
20311,Anand,White,Nunn John D M (ENG),2725.0,2615.0,Win,"It \""Melody Amber\"" (blindfold)",Monte Carlo (Monaco),1995.??.??,87.0


In [7]:
games =games.sort_values(by=['date','player'])


In [8]:
games=games.fillna(0)

In [9]:
games=games[games.opponent_Elo != 0]
games

,player,color,opponent,player_Elo,opponent_Elo,result,event,site,date,moves
16501,Botvinnik,White,Lilienthal Andor (HUN),2630.0,2385.0,Draw,It,Hastings (England),1934.??.??,69.0
16414,Botvinnik,Black,Lilienthal Andor (HUN),2630.0,2385.0,Draw,It,Moscow (Russia),1935.??.??,64.0
16063,Botvinnik,Black,Reshevsky Samuel H (USA),2630.0,2430.0,Draw,It,Nottingham (England),1936.??.??,81.0
16351,Botvinnik,White,Eliskases Erich G (AUT),2630.0,2430.0,Draw,It,Moscow (Russia),1936.??.??,114.0
16475,Botvinnik,White,Lilienthal Andor (HUN),2630.0,2385.0,Win,It,Moscow (Russia),1936.??.??,87.0
...,...,...,...,...,...,...,...,...,...,...
12989,Carlsen,White,"Nakamura, Hi",2842.0,2777.0,Win,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0
11930,Caruana,Black,"So, W.",2822.0,2780.0,Draw,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,70.0
5101,Nakamura,Black,"Carlsen, M.",2777.0,2842.0,Lose,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0
11881,Caruana,Black,"So, W.",2822.0,2780.0,Draw,Sinquefield GCT TB 2018,Saint Louis USA,2018.08.28,66.0


In [10]:
for i in range(15523) :
    if(games.iat[i,3]!=0):
        temp=games.iat[i,3]
    if(games.iat[i,3]==0):
        games.iloc[i, 3] = temp
games
        


,player,color,opponent,player_Elo,opponent_Elo,result,event,site,date,moves
16501,Botvinnik,White,Lilienthal Andor (HUN),2630.0,2385.0,Draw,It,Hastings (England),1934.??.??,69.0
16414,Botvinnik,Black,Lilienthal Andor (HUN),2630.0,2385.0,Draw,It,Moscow (Russia),1935.??.??,64.0
16063,Botvinnik,Black,Reshevsky Samuel H (USA),2630.0,2430.0,Draw,It,Nottingham (England),1936.??.??,81.0
16351,Botvinnik,White,Eliskases Erich G (AUT),2630.0,2430.0,Draw,It,Moscow (Russia),1936.??.??,114.0
16475,Botvinnik,White,Lilienthal Andor (HUN),2630.0,2385.0,Win,It,Moscow (Russia),1936.??.??,87.0
...,...,...,...,...,...,...,...,...,...,...
12989,Carlsen,White,"Nakamura, Hi",2842.0,2777.0,Win,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0
11930,Caruana,Black,"So, W.",2822.0,2780.0,Draw,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,70.0
5101,Nakamura,Black,"Carlsen, M.",2777.0,2842.0,Lose,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0
11881,Caruana,Black,"So, W.",2822.0,2780.0,Draw,Sinquefield GCT TB 2018,Saint Louis USA,2018.08.28,66.0


In [11]:
color = pd.get_dummies(games.color)
games = games.join(color)
games=games.drop(columns=['color'])
games

,player,opponent,player_Elo,opponent_Elo,result,event,site,date,moves,Black,White
16501,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,Draw,It,Hastings (England),1934.??.??,69.0,0,1
16414,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,Draw,It,Moscow (Russia),1935.??.??,64.0,1,0
16063,Botvinnik,Reshevsky Samuel H (USA),2630.0,2430.0,Draw,It,Nottingham (England),1936.??.??,81.0,1,0
16351,Botvinnik,Eliskases Erich G (AUT),2630.0,2430.0,Draw,It,Moscow (Russia),1936.??.??,114.0,0,1
16475,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,Win,It,Moscow (Russia),1936.??.??,87.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
12989,Carlsen,"Nakamura, Hi",2842.0,2777.0,Win,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0,0,1
11930,Caruana,"So, W.",2822.0,2780.0,Draw,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,70.0,1,0
5101,Nakamura,"Carlsen, M.",2777.0,2842.0,Lose,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0,1,0
11881,Caruana,"So, W.",2822.0,2780.0,Draw,Sinquefield GCT TB 2018,Saint Louis USA,2018.08.28,66.0,1,0


In [12]:
result= pd.get_dummies(games.result)
for i in range(15523) :
    if(result.iat[i,2]==1):
        games.iat[i,4]=int(2)
    if(result.iat[i,1]==1):
        games.iat[i,4]=int(0)
    if(result.iat[i,0]==1):
        games.iat[i,4]=int(1)
games

,player,opponent,player_Elo,opponent_Elo,result,event,site,date,moves,Black,White
16501,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,1,It,Hastings (England),1934.??.??,69.0,0,1
16414,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,1,It,Moscow (Russia),1935.??.??,64.0,1,0
16063,Botvinnik,Reshevsky Samuel H (USA),2630.0,2430.0,1,It,Nottingham (England),1936.??.??,81.0,1,0
16351,Botvinnik,Eliskases Erich G (AUT),2630.0,2430.0,1,It,Moscow (Russia),1936.??.??,114.0,0,1
16475,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,2,It,Moscow (Russia),1936.??.??,87.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
12989,Carlsen,"Nakamura, Hi",2842.0,2777.0,2,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0,0,1
11930,Caruana,"So, W.",2822.0,2780.0,1,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,70.0,1,0
5101,Nakamura,"Carlsen, M.",2777.0,2842.0,0,6th Sinquefield Cup 2018,Saint Louis USA,2018.08.27,193.0,1,0
11881,Caruana,"So, W.",2822.0,2780.0,1,Sinquefield GCT TB 2018,Saint Louis USA,2018.08.28,66.0,1,0


In [13]:
games.insert(0,'age',0)

In [14]:
for i in range(15523) :
    if(games.iloc[i,1]=='Anand'):
        games.iloc[i,0] = int(games.iloc[i,8][:4])-1969
    if(games.iloc[i,1]=='Botvinnik'):
        games.iloc[i,0] = int(games.iloc[i,8][:4])-1911
    if(games.iloc[i,1]=='Carlsen'):
        games.iloc[i,0] = int(games.iloc[i,8][:4])-1990
    if(games.iloc[i,1]=='caruana'):
        games.iloc[i,1] = int(games.iloc[i,8][:4])-1992
    if(games.iloc[i,1]=='Fischer'):
        games.iloc[i,0] = int(games.iloc[i,8][:4])-1943
    if(games.iloc[i,1]=='Kasparov'):
        games.iloc[i,0] = int(games.iloc[i,8][:4])-1963
    if(games.iloc[i,1]=='Nakamura'):
        games.iloc[i,0] = int(games.iloc[i,8][:4])-1987
    if(games.iloc[i,1]=='Polgar'):
        games.iloc[i,0] = int(games.iloc[i,8][:4])-1976
games['age']

16501    23
16414    24
16063    25
16351    25
16475    25
         ..
12989    28
11930     0
5101     31
11881     0
12332     0
Name: age, Length: 15523, dtype: int64

In [15]:
games=games[games.age > 0]
games=games.drop(columns=['date'])
games

,age,player,opponent,player_Elo,opponent_Elo,result,event,site,moves,Black,White
16501,23,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,1,It,Hastings (England),69.0,0,1
16414,24,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,1,It,Moscow (Russia),64.0,1,0
16063,25,Botvinnik,Reshevsky Samuel H (USA),2630.0,2430.0,1,It,Nottingham (England),81.0,1,0
16351,25,Botvinnik,Eliskases Erich G (AUT),2630.0,2430.0,1,It,Moscow (Russia),114.0,0,1
16475,25,Botvinnik,Lilienthal Andor (HUN),2630.0,2385.0,2,It,Moscow (Russia),87.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
14154,28,Carlsen,"Mamedyarov, S.",2842.0,2801.0,1,6th Sinquefield Cup 2018,Saint Louis USA,116.0,1,0
6073,31,Nakamura,"Karjakin, Sergey",2777.0,2773.0,1,6th Sinquefield Cup 2018,Saint Louis USA,83.0,0,1
18699,49,Anand,"Mamedyarov, S.",2768.0,2801.0,1,6th Sinquefield Cup 2018,Saint Louis USA,117.0,0,1
12989,28,Carlsen,"Nakamura, Hi",2842.0,2777.0,2,6th Sinquefield Cup 2018,Saint Louis USA,193.0,0,1


In [16]:
player = pd.get_dummies(games.player)
games = games.join(player)
games=games.drop(columns=['player'])
games

,age,opponent,player_Elo,opponent_Elo,result,event,site,moves,Black,White,Anand,Botvinnik,Carlsen,Fischer,Kasparov,Nakamura,Polgar
16501,23,Lilienthal Andor (HUN),2630.0,2385.0,1,It,Hastings (England),69.0,0,1,0,1,0,0,0,0,0
16414,24,Lilienthal Andor (HUN),2630.0,2385.0,1,It,Moscow (Russia),64.0,1,0,0,1,0,0,0,0,0
16063,25,Reshevsky Samuel H (USA),2630.0,2430.0,1,It,Nottingham (England),81.0,1,0,0,1,0,0,0,0,0
16351,25,Eliskases Erich G (AUT),2630.0,2430.0,1,It,Moscow (Russia),114.0,0,1,0,1,0,0,0,0,0
16475,25,Lilienthal Andor (HUN),2630.0,2385.0,2,It,Moscow (Russia),87.0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14154,28,"Mamedyarov, S.",2842.0,2801.0,1,6th Sinquefield Cup 2018,Saint Louis USA,116.0,1,0,0,0,1,0,0,0,0
6073,31,"Karjakin, Sergey",2777.0,2773.0,1,6th Sinquefield Cup 2018,Saint Louis USA,83.0,0,1,0,0,0,0,0,1,0
18699,49,"Mamedyarov, S.",2768.0,2801.0,1,6th Sinquefield Cup 2018,Saint Louis USA,117.0,0,1,1,0,0,0,0,0,0
12989,28,"Nakamura, Hi",2842.0,2777.0,2,6th Sinquefield Cup 2018,Saint Louis USA,193.0,0,1,0,0,1,0,0,0,0


In [17]:
site = pd.get_dummies(games.site)
site

,0,?,Aachen GER,Aalborg (Denmark),Abenra (Denmark),Achaea GRE,Adelaide (Australia),Aix-les-Bains FRA,Ajaccio FRA,Alma-Ata (Kazakhstan),...,Woburn (USA),Yerevan (Armenia),Yerevan ARM,Zalakaros (Hungary),Zaragoza (Spain),Zuerich SUI,Zug SUI,Zurich (Switzerland),Zurich SUI,chess.com INT
16501,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16063,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6073,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
games=games.drop(columns=['site','event','opponent'])
games.head()

,age,player_Elo,opponent_Elo,result,moves,Black,White,Anand,Botvinnik,Carlsen,Fischer,Kasparov,Nakamura,Polgar
16501,23,2630.0,2385.0,1,69.0,0,1,0,1,0,0,0,0,0
16414,24,2630.0,2385.0,1,64.0,1,0,0,1,0,0,0,0,0
16063,25,2630.0,2430.0,1,81.0,1,0,0,1,0,0,0,0,0
16351,25,2630.0,2430.0,1,114.0,0,1,0,1,0,0,0,0,0
16475,25,2630.0,2385.0,2,87.0,0,1,0,1,0,0,0,0,0


In [19]:
games["result"] = pd.to_numeric(games["result"])
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13568 entries, 16501 to 5101
Data columns (total 14 columns):
age             13568 non-null int64
player_Elo      13568 non-null float64
opponent_Elo    13568 non-null float64
result          13568 non-null int64
moves           13568 non-null float64
Black           13568 non-null uint8
White           13568 non-null uint8
Anand           13568 non-null uint8
Botvinnik       13568 non-null uint8
Carlsen         13568 non-null uint8
Fischer         13568 non-null uint8
Kasparov        13568 non-null uint8
Nakamura        13568 non-null uint8
Polgar          13568 non-null uint8
dtypes: float64(3), int64(2), uint8(9)
memory usage: 1.4 MB


In [20]:

y=games['result']
X=games.drop('result',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [21]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train,y_train)
predictions=dtc.predict(X_test)

In [22]:
predictions

array([1, 2, 2, ..., 0, 0, 1], dtype=int64)

In [23]:

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.22      0.25      0.23       617
           1       0.51      0.51      0.51      1712
           2       0.54      0.53      0.53      1742

    accuracy                           0.48      4071
   macro avg       0.42      0.43      0.43      4071
weighted avg       0.48      0.48      0.48      4071



In [24]:
confusion_matrix(y_test, predictions)

array([[152, 252, 213],
       [289, 866, 557],
       [246, 580, 916]], dtype=int64)

In [25]:
#מודל 1 אומר שכאשר יש פחות מ80 נקודות אז יהיה תיקו אחרת הגבוה ינצח
model1=0
for i in range(13568) :
    if(games.iloc[i, 1]-games.iloc[i, 2]>=80 and (games.iloc[i, 3]==2)):
        model1=model1+1
    if(games.iloc[i, 1]-games.iloc[i, 2]<=-80 and (games.iloc[i, 3]==0)):
        model1=model1+1
    if(games.iloc[i, 1]-games.iloc[i, 2]<80 and (games.iloc[i, 1]-games.iloc[i, 2]>-80)and(games.iloc[i, 3]==1)):
        model1=model1+1

In [26]:
model1/13568

0.5228478773584906

In [27]:
model0=0
for i in range(13568) :
    if(games.iloc[i, 1]-games.iloc[i, 2]>=10 and (games.iloc[i, 3]==2)):
        model0=model0+1
    if(games.iloc[i, 1]-games.iloc[i, 2]<=-10 and (games.iloc[i, 3]==0)):
        model0=model0+1
    if(games.iloc[i, 1]-games.iloc[i, 2]<10 and (games.iloc[i, 1]-games.iloc[i, 2]>-10)and(games.iloc[i, 3]==1)):
        model0=model0+1

In [28]:
model0/13568

0.44686025943396224

In [29]:
y=games['result']
X=games.drop('result',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X_train,y_train)
predictions=knn.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.34      0.23      0.27       624
           1       0.53      0.59      0.56      1728
           2       0.56      0.56      0.56      1719

    accuracy                           0.52      4071
   macro avg       0.47      0.46      0.46      4071
weighted avg       0.51      0.52      0.51      4071



In [30]:
confusion_matrix(y_test, predictions)

array([[ 142,  273,  209],
       [ 161, 1013,  554],
       [ 120,  631,  968]], dtype=int64)

In [31]:
#עץ החלטות
array([[152, 252, 213],
       [289, 866, 557],
       [246, 580, 916]]

SyntaxError: unexpected EOF while parsing (<ipython-input-31-294d52528c4b>, line 3)

TypeError: 'Series' object is not callable

In [38]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
predictions=rf.predict(X_test)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [39]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.27      0.21      0.24       624
           1       0.53      0.57      0.55      1728
           2       0.55      0.55      0.55      1719

    accuracy                           0.51      4071
   macro avg       0.45      0.44      0.45      4071
weighted avg       0.50      0.51      0.50      4071

